In [34]:
!pip install fredapi

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: C:\Users\rahul\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [35]:
!pip install python-dotenv

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: C:\Users\rahul\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [36]:
!nvidia-smi

NVIDIA-SMI has failed because you do not have suffient permissions. Please try running as an administrator.


In [37]:
import pandas as pd
import torch 
import matplotlib.pyplot as plt
from fredapi import Fred
import numpy as np

In [38]:
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.environ["API_KEY"]

In [39]:
fred = Fred(api_key=api_key)
data = fred.get_series("NROU")
data.tail()

2033-10-01    4.236570
2034-01-01    4.230920
2034-04-01    4.225190
2034-07-01    4.219381
2034-10-01    4.213503
dtype: float64

In [40]:
data = pd.DataFrame(data,columns=['Rate'])
data.reset_index(inplace=True)
data.head()

,index,Rate
0,1949-01-01,5.255053
1,1949-04-01,5.261516
2,1949-07-01,5.268013
3,1949-10-01,5.274564
4,1950-01-01,5.281182


In [41]:
data.rename(columns={"index": "Date"}, inplace=True)
data.head()

,Date,Rate
0,1949-01-01,5.255053
1,1949-04-01,5.261516
2,1949-07-01,5.268013
3,1949-10-01,5.274564
4,1950-01-01,5.281182


In [75]:
rates = data["Rate"].to_numpy()
rates_tensor = torch.from_numpy(rates).unsqueeze(dim=1)

In [76]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

In [82]:
cut_batch = int(len(rates_tensor) * .6)
end_batch = int(len(rates_tensor) * .2)

X_train = rates_tensor[:end_batch].type(torch.float32).to(device)
X_eval = rates_tensor[cut_batch:(cut_batch+ end_batch)].type(torch.float32).to(device)
X_test = rates_tensor[(cut_batch+ end_batch):].type(torch.float32).to(device)

len(X_train), len(X_eval), len(X_test)

(68, 68, 70)

In [83]:
X_train.dtype

torch.float32

In [84]:
import torch.nn as nn
class LinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear_layer = nn.Linear(in_features=1, out_features=1)

    def forward(self,x: torch.Tensor) -> torch.Tensor:
        return self.linear_layer(x)

In [85]:
model0 = LinearRegressionModel()

loss_fn = nn.L1Loss()

optimizer = torch.optim.SGD(model0.parameters(), lr=.01)

In [88]:
def noLossFr(epochs):
    for epoch in range(epochs):
        model0.train()
        x_pred = model0.forward(X_train)
        loss = loss_fn(x_pred, X_eval)
        print(loss)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

noLossFr(10)



tensor(1.4608, grad_fn=<MeanBackward0>)
tensor(1.1558, grad_fn=<MeanBackward0>)
tensor(0.8508, grad_fn=<MeanBackward0>)
tensor(0.5458, grad_fn=<MeanBackward0>)
tensor(0.2866, grad_fn=<MeanBackward0>)
tensor(0.2199, grad_fn=<MeanBackward0>)
tensor(0.2130, grad_fn=<MeanBackward0>)
tensor(0.2129, grad_fn=<MeanBackward0>)
tensor(0.2128, grad_fn=<MeanBackward0>)
tensor(0.2128, grad_fn=<MeanBackward0>)
